INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SANTA CATARINA \
CAMPUS FLORIANÓPOLIS \
DEPARTAMENTO ACADÊMICO DE ELETRÔNICA \
ENGENHARIA ELETRÔNICA

Estudante: 

# Atividade: Histograma e Binarização de Imagens

Existem diversos módulos de processamento de imagens para Python, como Pillow, OpenCV-Python, SimpleITK. Para esta atividade, recomendo o _scikit-image_ para a primeira parte e o OpenCV para a segunda. Vou mostrar um exemplo de como abrir uma imagem e gerar o histograma.

## Trabalhando com imagens pré-gravadas no computador
Aqui, você irá abrir uma imagem que está no disco, usando `skimage.io.imread`

In [ ]:
from skimage import io, exposure
import numpy as np
import matplotlib.pyplot as plt
fingerprint = io.imread('coins.jpg')
hist_y, hist_x = exposure.histogram(fingerprint)
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 12))
ax1.imshow(fingerprint, cmap=plt.cm.gray)
ax2.set_title('Histograma')
ax2.plot(hist_x, hist_y)
ax2.grid()
plt.show()

Nos itens seguintes, quando solicitado a mostrar a imagem resultante, use `imshowpair` [1] para mostrá-la lado a lado com a imagem original.

### Histograma

1. Compute o histograma da imagem _fingerprint_ [2] e apresente.

2. Na imagem _fingerprint_ [2], é possível distinguir o fundo e a impressão digital observando o histograma?

### Binarização

3. Qual limite poderia ser usado para binarizar a imagem *fingerprint*? Observe e apresente o histograma, teste e mostre a imagem binarizada resultante lado a lado com a imagem original usando `imshowpair`.

4. De forma semelhante, que limite poderia ser usado para binarizar a imagem _son1_ ou _darkerforeground_ [3]? Mostre o resultado.

5. Verifique que a binarização global da imagem _son1_ ou _darkerforeground_ [3] não provê um bom resultado. Use um limiar adaptativo (`threshold_local` no módulo _Scikit Image_ a partir da versão 0.13) [4] e mostre o resultado.

6. Observe o histograma da imagem _pout_ [5]. O que você pode concluir?

## Obtendo imagens a partir do ESP32-CAM
Embora seja possível processar as imagens adquiridas diretamente no ESP32, aqui usaremos um caminho misto. As imagens são capturadas pelo ESP32 e um cliente http as lê e faz o processamento local usando OpenCV no Python.

Então, a primeira parte é ter o servidor funcionando. Usaremos exatamente a mesma configuração da primeira atividade com a placa ESP32-CAM, disponível em https://github.com/fspacheco/esp32-cam/tree/main/ESP32-AP-CameraWebServer. Você pode observar no código em `app_httpd.cpp`, na função `startCameraServer()`, que existe um *uniform resource identifier* (URI) para **captura** de uma imagem, bastando apontar para http://192.168.4.1/capture e obtendo como retorno uma imagem no formato jpg.

Com o servidor funcionando, vamos abrir esse URI, ler a imagem e armazená-la em uma matriz (`frame` no código seguinte). Observe a dimensão dessa matriz com `.ndim`: como a imagem é colorida, há três planos de cores (RGB). É possível trabalhar com os três planos e obter um histograma para cada um deles, mas aqui vamos fazer a conversão para escala de cinza.

In [ ]:
# adaptado de https://how2electronics.com/color-detection-tracking-with-esp32-cam-opencv/
import cv2
import urllib.request
import numpy as np
 
url='http://192.168.4.1/capture'
 
while True:
    img_resp=urllib.request.urlopen(url)
    imgnp=np.array(bytearray(img_resp.read()),dtype=np.uint8)
    frame=cv2.imdecode(imgnp,-1)
    
    cv2.imshow("Imagem capturada", frame)
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    cv2.imshow("Imagem em tons de cinza", gray)
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27: # 27 - tecla ESC para sair
        break

# para gravar a última imagem, é só usar imwrite
cv2.imwrite('imagem-capturada.jpg', frame)
cv2.destroyAllWindows()
print('Núm. de planos da imagem: ', frame.ndim)
print('Dimensões da imagem: ', frame.shape)
print('Dimensões da imagem em tons de cinza: ', gray.shape)

7. Agora, a partir da imagem `gray`, use a função `cv.threshold` ou `cv.adaptiveThreshold` [6] para binarizar a imagem. Apresente o código e imagens resultantes.

Inclua, aqui, um quadro capturado e o resultado. Para isso, você deve gravar as imagens com `imwrite` e incorporar o link com o código mostrado abaixo. Ao enviar seu relatório, inclua as imagens no pacote zip.

Como exemplo, observe o que obtive com a minha câmera no ESP32-CAM.
![exemplo](imagem-capturada.jpg "Imagem capturada")

## Equalização de histograma (opcional)

8. Compare as técnicas de equalização de histograma implementadas nas funções `rescale_intensity`, `equalize_hist` e `equalize_adapthist` do _Scikit Image_ [7], usando a imagem _pout_ [5] como teste. Pesquise as referências apresentadas na documentação de cada uma das funções e explique as diferenças entre elas.

9. De forma semelhante, aplique uma das técnicas de equalização de histograma para realçar a imagem _dentalradiograph_ [8].

## Referências
[1] https://pypi.org/project/imshowpair/

[2] http://fmwconcepts.com/imagemagick/otsuthresh/images/fingerprint.jpg

[3] http://homepages.inf.ed.ac.uk/rbf/HIPR2/images/son1.gif ou https://www.mathworks.com/help/examples/images/win64/BinarizeImagesWithDarkerForegroundThanBackgroundExample_01.png

[4] https://scikit-image.org/docs/stable/auto_examples/applications/plot_thresholding.html

[5] http://www.johnloomis.org/ece563/notes/enhancement/scale/pout.jpg

[6] https://docs.opencv.org/4.x/d7/d4d/tutorial_py_thresholding.html

[7] https://scikit-image.org/docs/dev/auto_examples/color_exposure/plot_equalize.html

[8] https://imagej.nih.gov/ij/images/DentalRadiograph.png

[9] http://www.mathworks.com/help/releases/R2016b/examples/images/CompareResultsOfAveragingFilterAndMedianFilterExample_02.png


## Bibliografia
https://opensource.com/article/19/3/python-image-manipulation-tools

https://www.mathworks.com/examples/image/mw/images-ex28564424-create-image-histogram

https://www.mathworks.com/examples/image/mw/images-ex57959084-basic-image-enhancement-and-analysis-techniques

http://www.mathworks.com/help/images/ref/imbinarize.html

http://blogs.mathworks.com/steve/2016/06/14/image-binarization-otsus-method

https://www.mathworks.com/examples/image/mw/images-ex97343802-contrast-enhancement-techniques

https://www.mathworks.com/examples/image/mw/images-ex74217292-remove-noise-using-an-averaging-filter-and-a-median-filter

https://www.mathworks.com/examples/image/mw/images-ex48835658-detect-edges-in-images

https://www.mathworks.com/examples/image/mw/images-ex80028725-trace-boundaries-of-objects-in-images

https://www.mathworks.com/examples/image/mw/images-ex45754758-filter-images-using-predefined-filters

https://www.mathworks.com/examples/image/community/18507-missing-planes

https://how2electronics.com/color-detection-tracking-with-esp32-cam-opencv/